In [4]:
from io import StringIO
from elasticsearch import Elasticsearch, helpers
from datetime import datetime
import re
import pandas as pd
import openpyxl 
import numpy as np
import os

pd.set_option('display.max_columns', 100)

In [5]:
#connect es
es = Elasticsearch([{'host': 'localhost', 'port':9200}])

In [6]:
#get all indices in es
sorted(list(es.indices.get_alias('*').keys()))

['.apm-agent-configuration',
 '.kibana_1',
 '.kibana_task_manager_1',
 '.security-7',
 'account_db',
 'assignedreadonly',
 'audit_trail',
 'contact_db',
 'detail_ctr_2013',
 'detail_ctr_2014',
 'detail_ctr_2015',
 'detail_ctr_2016',
 'detail_ctr_2017',
 'detail_ctr_2018',
 'detail_ctr_2019',
 'detail_ctr_2020',
 'detail_itis',
 'detail_str',
 'distdates',
 'distratio2020',
 'distrecords',
 'entity_db',
 'itis_db',
 'leave',
 'removedrecord',
 'report_db',
 'tax']

### STR

### 22/01

In [20]:
#query example
indexName = 'detail_str'

startDate = "01/01/2017"
endDate = "31/12/2019"
queryStr = {"bool":{"must":[{"range":{"reported_date":{"gte":startDate, "lte":endDate}}}]}}
# queryStr = {'match_all':{}}
results = helpers.scan(es, index = indexName, query = {'query':queryStr})

temp = []
for i in results:
    temp.append(i['_source'])

In [21]:
len(temp)

263867

In [147]:
1) same + description contains ['order', 'perintah'] -  number
2) reactive = "yes" - number

dict_keys(['record_id', 'name_list', 'id_list', 'contact_list', 'summary', 'detail', 'reported_date', 'trx_date', 'suspicion_desc', 'reactive', 'uid'])

### Details of all STRs reported by DNFBP industry players relating to investigation orders issued by LEAs to them or their customers.

In [22]:
infoList = []
for i in temp:
    tempDict = i['detail']['report']
    tempDict.update({'suspicion_desc':i['suspicion_desc']})
    tempDict.update({'reactive':i['reactive']})
    infoList.append(tempDict)

In [23]:
len(infoList)

263867

In [197]:
#not all str has ground_suspicion
sum([True for i in infoList if "GROUND_SUSPICION" in i])

263428

In [87]:
gsTypeList = [i["GROUND_SUSPICION"] for i in infoList if "GROUND_SUSPICION" in i]

#just get str that has ground_suspicion in its field
groundList2 = [i for i in infoList if "GROUND_SUSPICION" in i]

#ground suspicion that contains order
groundList3 = [i for i in groundList2 if re.search("order", i['GROUND_SUSPICION'], flags = re.I)]

In [200]:
len(groundList3)

14753

In [185]:
df = pd.DataFrame(groundList3)

df['ORG_ID2'] = df['RECORD_ID'].map(lambda x:x.split('/')[0] + '/' + x.split('/')[1])

In [25]:
#read org list to get org list categorized under dfnbp
excelFile = 'org.csv'
dfOrg = pd.read_csv(excelFile)

In [186]:
#sectors under dnfbp
sectorList = ["Casino", "Gaming Outlet Operator", "Jewellers & Precious Stones/Metals Dealers",
             "Building Credit Business", "Trust Company", "Company Secretary",
             "Lawyer", "Accountant", "Moneylender and Pawnbroker",
             "Real Estate Agent", "Leasing and Factoring"]
#add takaful - DA/003
orgIDList = list(dfOrg.query("ORG_GROUP_NAME in @sectorList")['ORG_ID'].values) + ['DA/003']

#filter ground_suspicion contains order
df_DFNBP = df[df['ORG_ID2'].map(lambda x:x in orgIDList)].copy()

In [187]:
df_DFNBP.shape

(112, 38)

In [225]:
a = df_DFNBP['RECORD_ID'].unique()

In [212]:
#just get str that has ground_suspicion in its field or description contains ['order', 'perintah']
filterList = [i for i in infoList if "GROUND_SUSPICION" in i or 
               re.search("order|perintah", i['suspicion_desc'], flags = re.I)]

In [214]:
dfFilter = pd.DataFrame(filterList)

(263479, 43)

In [215]:
dfFilter['ORG_ID2'] = dfFilter['RECORD_ID'].map(lambda x:x.split('/')[0] + '/' + x.split('/')[1])

In [219]:
#sectors under dnfbp
sectorList = ["Casino", "Gaming Outlet Operator", "Jewellers & Precious Stones/Metals Dealers",
             "Building Credit Business", "Trust Company", "Company Secretary",
             "Lawyer", "Accountant", "Moneylender and Pawnbroker",
             "Real Estate Agent", "Leasing and Factoring"]
#add takaful - DA/003
orgIDList = list(dfOrg.query("ORG_GROUP_NAME in @sectorList")['ORG_ID'].values) + ['DA/003']

#filter ground_suspicion contains order
dfFilterFinal = dfFilter[dfFilter['ORG_ID2'].map(lambda x:x in orgIDList)].copy()

In [220]:
dfFilterFinal.shape

(22853, 44)

In [27]:
#just get str that reactive = yes
filterList2 = [i for i in infoList if i['reactive'] == 'Yes']
len(filterList2)

42646

In [28]:
dfFilter2 = pd.DataFrame(filterList2)
dfFilter2['ORG_ID2'] = dfFilter2['RECORD_ID'].map(lambda x:x.split('/')[0] + '/' + x.split('/')[1])

In [29]:
#sectors under dnfbp
sectorList = ["Casino", "Gaming Outlet Operator", "Jewellers & Precious Stones/Metals Dealers",
             "Building Credit Business", "Trust Company", "Company Secretary",
             "Lawyer", "Accountant", "Moneylender and Pawnbroker",
             "Real Estate Agent", "Leasing and Factoring"]
#add takaful - DA/003
orgIDList = list(dfOrg.query("ORG_GROUP_NAME in @sectorList")['ORG_ID'].values) + ['DA/003']

#filter ground_suspicion contains order
dfFilterFinal2 = dfFilter2[dfFilter2['ORG_ID2'].map(lambda x:x in orgIDList)].copy()

In [33]:
dfFilterFinal2.head()

,ORG_ID,RECORD_ID,RECORD_TYPE_ID,ATTEMPTED_FLAG,STATE_ID,BRANCH_ID,ACCOUNT_TYPE_ID,FROM_TRX_DATE,TO_TRX_DATE,TRX_AMOUNT,CURRENCY_ID1,CURRENCY_ID2,REPORTED_DATE,GROUND_SUSPICION,OFFENCE_TYPE_ID,OTHER_SUSPICION,suspicion_desc,reactive,ACCOUNT_NO,ACCOUNT_STATUS_ID,TRX_TYPE_ID,ACCOUNT_OPEN_DATE,ACCOUNT_BALANCE,AMOUNT_FOREIGN1,CDS_NO,PRODUCT_CODE,COUNTER_CODE,TOTAL_FUND,POLICY_NO,POLICY_TYPE,INSURANCE_TYPE_ID,BUSINESS_CLASS_ID,POLICY_COMM_DATE,SUM_INSURED,PREMIUM_AMOUNT,CLAIM_NO,MEMBER_NO,MEMBER_TYPE,JOINED_DATE,ORG_ID2
41653,Orix Leasing Malaysia,AO/005/S/2018/000001F,STR Leasing and Factoring,N,UNKNOWN,UNKNOWN,UNKNOWN,01/11/2018,13/11/2018,28292.0,NaN,NaN,23/11/2018,Others (Please specify),Money Laundering,SETTLEMENT OF CONTRACTS BY 3RD PARTY AS CUSTOM...,REFER TO ANNEXURE,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AO/005
41691,MY Bullion Trade,AQ/006/S/2019/000001F,STR Jewellers & Precious Stones/Metals Dealers,N,UNKNOWN,UNKNOWN,UNKNOWN,10/04/2019,15/04/2019,436000.0,NaN,NaN,16/04/2019,Unwillingness of buyer to disclose identity,Fraud,"PURCHASER LOOKS SUSPICIOUS, SHIFTY EYES AND DO...",AS PER ATTACHED POLICE REPORT.,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AQ/006
41692,Perpetual Trustees Berhad,AS/004/S/2017/000001F,STR Trust Companies,N,NON-MALAYSIAN STATE,UNKNOWN,OTHERS,04/05/2017,NaN,0.0,NaN,NaN,04/05/2017,"Counterfeit/ fraud/ unauthorised transaction, ...",Unknown,AS PER POLICE REPORT ATTACHED,DETAILS IN ATTACHED POLICE REPORT,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AS/004
41693,Kenanga Capital Sdn Bhd.,AT/001/S/2019/000001,STR Moneylender and Pawnbroker,N,UNKNOWN,UNKNOWN,UNKNOWN,18/02/2019,18/02/2019,0.0,NaN,NaN,21/02/2019,"Investigation Order - MACC, Investigation Orde...",Money Laundering,PREVIOUSLY REPRIMANDED BY MACC,A) CLIENT'S INFORMATION\r\r\n\r\r\n 1. THE CLI...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AT/001
41804,Resorts World Bhd.,DA/002/S/2017/000281,STR Casino,N,UNKNOWN,UNKNOWN,UNKNOWN,02/11/2016,02/11/2016,0.0,NaN,NaN,10/01/2017,Investigation Order - Others,Tax Offences,SEKSYEN 114 AKTA CUKAI PENDAPATAN 1967,"ON 02 NOVEMBER 2016, GENTING MALAYSIA BERHAD (...",Yes,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88085024,Platinum Principal,11/08/1999,DA/002


In [32]:
csvFile = "2021-01-25_ReactiveStrDFNBP.csv"
dfFilterFinal2.to_csv(csvFile, index = False)

In [226]:
b = dfFilterFinal2['RECORD_ID'].unique()

In [227]:
len(set(b)&set(a))

112

In [229]:
c = set(b).difference(set(a))
dfFilterFinal3 = dfFilterFinal2[dfFilterFinal2['RECORD_ID'].isin(c)]
dfFilterFinal3.shape

(55, 40)

In [230]:
csvFile = "2021-01-22_55ReactiveStr.csv"
dfFilterFinal3.to_csv(csvFile, index = False)

In [145]:
csvFile = "2021-01-22_StrTableDFNBP.csv"
df_DFNBP.to_csv(csvFile, index = False)

### details of all STRs reported by gaming institutions, covering period from year 2017 to year 2019

In [202]:
sectorList = ["Gaming Outlet Operator"]
orgIDList = dfOrg.query("ORG_GROUP_NAME in @sectorList")['ORG_ID'].values

In [206]:
infoList[0]

{'ORG_ID': 'Spectrum Forex Sdn. Bhd.',
 'RECORD_ID': '117/271800V/S/2017/000004',
 'RECORD_TYPE_ID': 'STR Money changer',
 'ATTEMPTED_FLAG': 'N',
 'STATE_ID': 'UNKNOWN',
 'BRANCH_ID': 'UNKNOWN',
 'ACCOUNT_TYPE_ID': 'UNKNOWN',
 'FROM_TRX_DATE': '19/11/2016',
 'TO_TRX_DATE': '25/11/2016',
 'TRX_AMOUNT': '36746.0',
 'CURRENCY_ID1': 'US Dollar',
 'CURRENCY_ID2': 'Malaysian Ringgit',
 'REPORTED_DATE': '18/01/2017',
 'GROUND_SUSPICION': 'Others (Please specify)',
 'OFFENCE_TYPE_ID': 'Bribery/Corruption',
 'OTHER_SUSPICION': 'REFUSED TO GIVE DOCUMENT (TRANSACTION ON BEHALF OF ANOTHER PERSON)',
 'suspicion_desc': 'CUSTOMER, MURUGAYA A/L SUBBIAH, 50 (D.O.B: 10 JAN 1967) EMPLOYED AS A DRIVER.\r\r\n\r\r\nCUSTOMER HAS CAME 3 TIMES IN NILAI TESCO BRANCH TO PERFORM MONEY CHANGING TRANSACTIONS, DETAILS OF TRANSACTION AS BELOW:\r\r\n\r\r\n      DATE\t          TRAN TIME\t      B/S            CURR\tAMOUNT (RM)\r\r\n25-NOV-16\t4:10:00 PM\t   WESELL\tUSD\tRM4,460.00\r\r\n19-NOV-16\t2:15:00 PM\t   WEBUY\t

In [201]:
df2 = pd.DataFrame(infoList)
df2.shape

(263867, 42)

In [203]:
df2['ORG_ID2'] = df2['RECORD_ID'].map(lambda x:x.split('/')[0] + '/' + x.split('/')[1])
df_DFNBP2 = df2[df2['ORG_ID2'].map(lambda x:x in orgIDList)].copy()

In [204]:
df_DFNBP2.shape

(261, 43)

In [209]:
csvFile = "2021-01-22_StrGamingInst.csv"
df_DFNBP2.to_csv(csvFile, index = False)

### 22/02

In [7]:
#query example
indexName = 'detail_str'

startDate = "01/01/2020"
endDate = "31/12/2020"
queryStr = {"bool":{"must":[{"range":{"reported_date":{"gte":startDate, "lte":endDate}}}]}}
# queryStr = {'match_all':{}}
results = helpers.scan(es, index = indexName, query = {'query':queryStr})

temp = []
for i in results:
    temp.append(i['_source'])

In [8]:
len(temp)

76075

In [9]:
infoList = []
for i in temp:
    tempDict = i['detail']['report']
    tempDict.update({'suspicion_desc':i['suspicion_desc']})
    tempDict.update({'reactive':i['reactive']})
    infoList.append(tempDict)

df = pd.DataFrame(infoList)

df['ORG_ID2'] = df['RECORD_ID'].map(lambda x:x.split('/')[0] + '/' + x.split('/')[1])

In [17]:
df['REPORTED_DATE'].unique()

array(['13/01/2020', '07/01/2020', '03/01/2020', '08/01/2020',
       '09/01/2020', '11/01/2020', '04/01/2020', '12/01/2020',
       '06/01/2020', '02/01/2020', '10/01/2020', '05/01/2020',
       '01/01/2020', '17/01/2020', '12/02/2020', '13/02/2020',
       '16/01/2020', '20/01/2020', '21/01/2020', '23/01/2020',
       '06/02/2020', '17/02/2020', '20/02/2020', '31/01/2020',
       '28/01/2020', '22/01/2020', '24/01/2020', '04/02/2020',
       '14/02/2020', '05/02/2020', '03/02/2020', '18/02/2020',
       '29/01/2020', '27/01/2020', '08/02/2020', '15/01/2020',
       '07/02/2020', '19/02/2020', '30/01/2020', '14/01/2020',
       '19/01/2020', '21/02/2020', '11/02/2020', '01/02/2020',
       '09/02/2020', '22/02/2020', '02/02/2020', '10/02/2020',
       '25/01/2020', '26/01/2020', '18/01/2020', '16/02/2020',
       '23/02/2020', '15/02/2020', '06/03/2020', '03/03/2020',
       '28/02/2020', '18/03/2020', '04/03/2020', '09/03/2020',
       '11/03/2020', '16/03/2020', '08/04/2020', '10/04

In [10]:
#read org list to get org list categorized under dfnbp
excelFile = 'data/raw/org.csv'
dfOrg = pd.read_csv(excelFile)

In [11]:
#sectors under dnfbp
sectorList = ["Casino", "Gaming Outlet Operator", "Jewellers & Precious Stones/Metals Dealers",
             "Building Credit Business", "Trust Company", "Company Secretary",
             "Lawyer", "Accountant", "Moneylender and Pawnbroker",
             "Real Estate Agent", "Leasing and Factoring"]
#add takaful - DA/003
orgIDList = list(dfOrg.query("ORG_GROUP_NAME in @sectorList")['ORG_ID'].values) + ['DA/003']

In [12]:
df_DFNBP = df[df['ORG_ID2'].map(lambda x:x in orgIDList)].copy()

In [13]:
df_DFNBP.columns

Index(['ORG_ID', 'RECORD_ID', 'RECORD_TYPE_ID', 'ATTEMPTED_FLAG', 'STATE_ID',
       'BRANCH_ID', 'ACCOUNT_TYPE_ID', 'FROM_TRX_DATE', 'TO_TRX_DATE',
       'TRX_AMOUNT', 'CURRENCY_ID1', 'CURRENCY_ID2', 'REPORTED_DATE',
       'GROUND_SUSPICION', 'OFFENCE_TYPE_ID', 'OTHER_SUSPICION',
       'suspicion_desc', 'reactive', 'ACCOUNT_NO', 'ACCOUNT_STATUS_ID',
       'TRX_TYPE_ID', 'ACCOUNT_OPEN_DATE', 'ACCOUNT_BALANCE',
       'AMOUNT_FOREIGN1', 'CDS_NO', 'PRODUCT_CODE', 'COUNTER_CODE',
       'TOTAL_FUND', 'POLICY_NO', 'POLICY_TYPE', 'INSURANCE_TYPE_ID',
       'BUSINESS_CLASS_ID', 'POLICY_COMM_DATE', 'SUM_INSURED',
       'PREMIUM_AMOUNT', 'MEMBER_NO', 'MEMBER_TYPE', 'JOINED_DATE',
       'WINNING_AMOUNT', 'TICKET_NO', 'REGISTRATION_NO', 'MONEY_ORDER_NO',
       'CLAIM_NO', 'ORG_ID2'],
      dtype='object')

In [14]:
df_DFNBP['RECORD_TYPE_ID'].unique()

array(['STR Jewellers & Precious Stones/Metals Dealers', 'STR Casino',
       'STR financial', 'STR Leasing and Factoring',
       'STR Moneylender and Pawnbroker', 'STR Gaming Outlet',
       'STR Company Secretary', 'STR Lawyer-Accountant',
       'STR Trust Companies'], dtype=object)

In [16]:
df_DFNBP['REPORTED_DATE'].unique()

array(['03/01/2020', '08/01/2020', '09/01/2020', '02/01/2020',
       '06/01/2020', '07/01/2020', '10/01/2020', '01/01/2020',
       '13/01/2020', '21/01/2020', '13/02/2020', '20/02/2020',
       '17/01/2020', '14/01/2020', '16/01/2020', '20/01/2020',
       '23/01/2020', '22/01/2020', '24/01/2020', '12/02/2020',
       '31/01/2020', '06/02/2020', '29/01/2020', '03/02/2020',
       '30/01/2020', '04/02/2020', '07/02/2020', '10/02/2020',
       '05/02/2020', '17/02/2020', '11/02/2020', '14/02/2020',
       '18/02/2020', '19/02/2020', '21/02/2020', '30/04/2020',
       '16/03/2020', '03/03/2020', '27/02/2020', '28/02/2020',
       '24/02/2020', '05/03/2020', '02/03/2020', '25/02/2020',
       '26/02/2020', '04/03/2020', '09/03/2020', '06/03/2020',
       '10/03/2020', '12/03/2020', '11/03/2020', '13/03/2020',
       '17/03/2020', '01/04/2020', '31/03/2020', '03/04/2020',
       '04/04/2020', '18/03/2020', '24/03/2020', '28/03/2020',
       '02/04/2020', '05/04/2020', '08/04/2020', '06/04

In [20]:
csvFile = "2021-02-22_StrTableDNFBP2020.csv"
df_DFNBP.to_csv(csvFile, index = False)

In [18]:
csvFile = "2021-02-22_StrTableDNFBP2020.csv"
df = pd.read_csv(csvFile)

In [20]:
df['RECORD_TYPE_ID'].unique()

array(['STR Jewellers & Precious Stones/Metals Dealers',
       'STR Trust Companies', 'STR Casino', 'STR financial',
       'STR Gaming Outlet', 'STR Lawyer-Accountant',
       'STR Leasing and Factoring', 'STR Moneylender and Pawnbroker',
       'STR Company Secretary'], dtype=object)

### ITIS

In [4]:
#query example
indexName = 'detail_itis'

startDate = "01/01/2016"
endDate = "31/12/2020"
queryStr = {"bool":{"must":[{"range":{"reported_date":{"gte":startDate, "lte":endDate}}}]}}
# queryStr = {'match_all':{}}
results = helpers.scan(es, index = indexName, query = {'query':queryStr})

temp = []
for i in results:
    temp.append(i['_source'])

In [5]:
len(temp)

5650627

In [23]:
temp[0]

{'reported_date': '16/02/2016',
 'id_list': ['993229W'],
 'name_list': ['SABAH SHELL PETROLEUM'],
 'detail': {'RESIDENT_NAME': 'SABAH SHELL PETROLEUM',
  'RESIDENT_ID': '993229W',
  'NON_RESIDENT_COUNTRY': 'INDONESIA',
  'REPORTED_DATE': '16/02/2016',
  'ORG_ID': 'RHB BANK BERHAD',
  'TRANSACTION_TYPE': 'Receipts',
  'AMOUNT': '9365305.64',
  'TRANSACTION_PURPOSE': 'HEAD OFFICE ACCOUNTS IN BRANCHES'},
 'uid': ['e-64506a9e-3e76-46bb-8891-72849f5bc790']}

In [6]:
cleanTemp = []
for i in temp:
    tempDict = {}
    for key in i:
        if key == "detail":
            for key2 in i[key]:
                tempDict[key2] = i[key][key2]
        elif key not in ['uid']:
            tempDict[key] = i[key]
    cleanTemp.append(tempDict)

In [7]:
len(cleanTemp)

5650627

In [8]:
dfItis = pd.DataFrame(cleanTemp)

In [16]:
dfItis.columns

Index(['reported_date', 'id_list', 'name_list', 'RESIDENT_NAME', 'RESIDENT_ID',
       'NON_RESIDENT_COUNTRY', 'REPORTED_DATE', 'ORG_ID', 'TRANSACTION_TYPE',
       'AMOUNT', 'TRANSACTION_PURPOSE', 'NON_RESIDENT_NAME', 'account',
       'ACCOUNT_NO', 'is_tagged'],
      dtype='object')

In [8]:
IdList = dfItis['RESIDENT_ID'].unique()

In [9]:
len(IdList)

126299

In [9]:
dfItis['Year'] = pd.to_datetime(dfItis['reported_date']).dt.year

dfItis['AMOUNT'] = dfItis['AMOUNT'].astype(float)

In [34]:
dfItis.columns

Index(['reported_date', 'id_list', 'name_list', 'RESIDENT_NAME', 'RESIDENT_ID',
       'NON_RESIDENT_COUNTRY', 'REPORTED_DATE', 'ORG_ID', 'TRANSACTION_TYPE',
       'AMOUNT', 'TRANSACTION_PURPOSE', 'NON_RESIDENT_NAME', 'account',
       'ACCOUNT_NO', 'is_tagged', 'Year'],
      dtype='object')

In [40]:
dfItis['Year'].value_counts()

2019    1416231
2018    1116927
2017    1066588
2020    1061649
2016     989232
Name: Year, dtype: int64

In [10]:
excelFile = "data/raw/Data for ITIS - LFSA January 2021.xlsx"
wb_obj = openpyxl.load_workbook(excelFile) 

sheetNameList = wb_obj.sheetnames

In [131]:
entityIdList = [k for i,j in entitiesDict2.items() for k in j]

In [125]:
excelFile = "data/raw/Data  - STR, CTRs & ITIS  - LIBFC RA 2020.xlsx"
dfIDYL = pd.read_excel(excelFile)

idListYL = [re.sub("/","",i) for i in dfIDYL['LL Number'].values]

In [132]:
len(idListYL), len(entityIdList)

(6210, 6210)

In [133]:
len(set(idListYL) & set(entityIdList))

6176

In [135]:
set(idListYL).difference(set(entityIdList))

{'LAF00087-(I)',
 'LAF00206-(I)',
 'LAF00284-(I)',
 'LAF00285-(I)',
 'LAF00333-(I)',
 'LAF00399-(I)',
 'LLP001432015-(I)',
 'LLP001812015-(I)',
 'LLP001842015-(I)',
 'LLP001852015-(I)',
 'LLP00462013-(I)'}

In [136]:
set(entityIdList).difference(set(idListYL))

{'LAF00087(I)',
 'LAF00206(I)',
 'LAF00284(I)',
 'LAF00285(I)',
 'LAF00333(I)',
 'LAF00399(I)',
 'LLP001432015(I)',
 'LLP001812015(I)',
 'LLP001842015(I)',
 'LLP001852015(I)',
 'LLP00462013(I)'}

In [12]:
entitiesDict = {}
for sector in sheetNameList:
    df = pd.read_excel(excelFile, sheet_name = sector)
    entitiesDict[sector] = list(df['Column1'].values)

In [13]:
#remove slash symbol for partnerships, trusts
entitiesDict2 = {i:[re.sub('/|-', '',x) for x in j] for i,j in entitiesDict.items()}

In [14]:
resultList = []
for i in ['Receipts', 'Payments']:
    for year in np.arange(2016, 2021):
        for sector in entitiesDict2:
            dfTemp = dfItis[(dfItis['TRANSACTION_TYPE'] == i) &
                           (dfItis['Year'] == year) &
                           (dfItis['RESIDENT_ID'].isin(entitiesDict2[sector]))].copy()
            sumAmount = dfTemp['AMOUNT'].sum()
            noEntity = len(dfTemp['RESIDENT_ID'].unique())
            noTrans = dfTemp.shape[0]
            #store result: Sector, Receipts/Payments, Year, TotalAmount, NoPlayer 
            tempList = [sector, i, year, sumAmount, noEntity, noTrans]
            resultList.append(tempList)

In [95]:
dfResult = pd.DataFrame(resultList)
dfResult.columns = ["Sector", "TransactionType", "Year", "TotalAmount", "NoPlayer", "NoTrans"]
del dfResult['NoPlayer']
dfResult

,Sector,TransactionType,Year,TotalAmount,NoTrans
0,Banks,Receipts,2016,0.000000e+00,0
1,Insurance,Receipts,2016,0.000000e+00,0
2,Leasing,Receipts,2016,0.000000e+00,0
3,LITC,Receipts,2016,2.220020e+08,22
4,Money Broker,Receipts,2016,0.000000e+00,0
...,...,...,...,...,...
105,Trust Company,Payments,2020,0.000000e+00,0
106,Foundations,Payments,2020,0.000000e+00,0
107,Labuan Companies,Payments,2020,0.000000e+00,0
108,Partnerships,Payments,2020,0.000000e+00,0


In [96]:
csvFile = "data/interim/2021-01-30_ItisEsTempTable.csv"
dfResult.to_csv(csvFile, index = False)

### 1st table: Total of player by sector and year

In [16]:
dfAgg = dfResult.groupby(['Year', 'TransactionType', 'Sector'])['NoTrans'].sum().reset_index()
dfAgg.head(10)

,Year,TransactionType,Sector,NoTrans
0,2016,Payments,Banks,0
1,2016,Payments,Foundations,0
2,2016,Payments,Insurance,1
3,2016,Payments,LITC,23
4,2016,Payments,Labuan Companies,6
5,2016,Payments,Leasing,0
6,2016,Payments,Money Broker,0
7,2016,Payments,Partnerships,0
8,2016,Payments,Securities Licensee,0
9,2016,Payments,Trust Company,0


In [18]:
dfFinal1 = pd.pivot_table(dfAgg, values = "NoTrans", index = ['TransactionType', 'Sector'], 
               columns = ['Year'], aggfunc=np.sum)
dfFinal1

Year                                 2016  2017  2018  2019  2020
TransactionType Sector                                           
Payments        Banks                   0     0     0     0     0
                Foundations             0     0     0     0     0
                Insurance               1     2     0     0     0
                LITC                   23     0     0     0     0
                Labuan Companies        6     4     0     0     0
                Leasing                 0     0     0     0     0
                Money Broker            0     0     0     0     0
                Partnerships            0     0     0     0     0
                Securities Licensee     0     0     0     0     0
                Trust Company           0     0     0     0     0
                Trusts                  0     0     0     0     0
Receipts        Banks                   0     0     0     0     0
                Foundations             0     0     0     0     0
                Insurance               0     2     0     0     0
                LITC                   22     0     0     0     0
                Labuan Companies        0     7     0     0     0
                Leasing                 0     1     0     0     0
                Money Broker            0     0     0     0     0
                Partnerships            0     0     0     0     0
                Securities Licensee     0     0     0     0     0
                Trust Company           0     0     0     0     0
                Trusts                  0     0     0     0     0

### 2nd table: Total Amount by sector, transaction type and year

In [19]:
# dfResult['Type'] = dfResult['TransactionType'].map(lambda x: "Outwards" if x == "Payments"
#                                                   else "Inwards")

In [22]:
dfFinal2 = pd.pivot_table(dfResult, values = "TotalAmount", index = ['TransactionType', 'Sector'], 
                           columns = ['Year'], aggfunc=np.sum)
dfFinal2

Year                                         2016         2017  2018  2019  \
TransactionType Sector                                                       
Payments        Banks                0.000000e+00         0.00   0.0   0.0   
                Foundations          0.000000e+00         0.00   0.0   0.0   
                Insurance            6.799250e+05   2406318.07   0.0   0.0   
                LITC                 3.231467e+08         0.00   0.0   0.0   
                Labuan Companies     4.307169e+06  15558019.11   0.0   0.0   
                Leasing              0.000000e+00         0.00   0.0   0.0   
                Money Broker         0.000000e+00         0.00   0.0   0.0   
                Partnerships         0.000000e+00         0.00   0.0   0.0   
                Securities Licensee  0.000000e+00         0.00   0.0   0.0   
                Trust Company        0.000000e+00         0.00   0.0   0.0   
                Trusts               0.000000e+00         0.00   0.0   0.0   
Receipts        Banks                0.000000e+00         0.00   0.0   0.0   
                Foundations          0.000000e+00         0.00   0.0   0.0   
                Insurance            0.000000e+00    812567.09   0.0   0.0   
                LITC                 2.220020e+08         0.00   0.0   0.0   
                Labuan Companies     0.000000e+00  14154097.37   0.0   0.0   
                Leasing              0.000000e+00   1544797.92   0.0   0.0   
                Money Broker         0.000000e+00         0.00   0.0   0.0   
                Partnerships         0.000000e+00         0.00   0.0   0.0   
                Securities Licensee  0.000000e+00         0.00   0.0   0.0   
                Trust Company        0.000000e+00         0.00   0.0   0.0   
                Trusts               0.000000e+00         0.00   0.0   0.0   

Year                                 2020  
TransactionType Sector                     
Payments        Banks                 0.0  
                Foundations           0.0  
                Insurance             0.0  
                LITC                  0.0  
                Labuan Companies      0.0  
                Leasing               0.0  
                Money Broker          0.0  
                Partnerships          0.0  
                Securities Licensee   0.0  
                Trust Company         0.0  
                Trusts                0.0  
Receipts        Banks                 0.0  
                Foundations           0.0  
                Insurance             0.0  
                LITC                  0.0  
                Labuan Companies      0.0  
                Leasing               0.0  
                Money Broker          0.0  
                Partnerships          0.0  
                Securities Licensee   0.0  
                Trust Company         0.0  
                Trusts                0.0

### NR to NR

In [34]:
excelFolder = "ITIS_NR-and-NR/data"
pandasList = []
for file in os.listdir(excelFolder):
    df = pd.read_excel(excelFolder + '/' + file)
    pandasList.append(df)

In [35]:
dfITIS2 = pd.concat(pandasList).copy()

In [36]:
dfITIS2.shape

(3005151, 16)

In [42]:
ITIS2idList = dfITIS2['NR Customer Details.Customer Beneficiary Remitter Identification Number'].unique()

In [61]:
ITIS2idList2 = dfITIS2['NR Transaction Details.Beneficiary Remitter Identification Number'].unique()

In [45]:
len(ITIS2idList)

709796

In [62]:
len(ITIS2idList2)

798064

In [53]:
entityIdList = [k for i,j in entitiesDict2.items() for k in j]
len(entityIdList)

6210

In [57]:
a = []
for i in ITIS2idList:
    try:
        if str(i) in entityIdList:
            a.append(i)
    except:
        print(i)
        break

In [58]:
len(a)

2781

In [63]:
len([i for i in ITIS2idList2 if str(i) in entityIdList])

1076

In [41]:
dfITIS2['NR Transaction Details.Transaction Type'].unique()

array(['Other Payments', 'Other Receipts', 'Unknown'], dtype=object)

In [38]:
dfITIS2['Reporting Period.Reporting Year'].unique()

array([2017, 2018, 2019, 2020], dtype=int64)

In [67]:
resultList2 = []
resultList3 = []
amountCol = 'NR - MYR Amount'
transTypeCol = 'NR Transaction Details.Transaction Type'
yearCol = 'Reporting Period.Reporting Year'
idCol1 = 'NR Customer Details.Customer Beneficiary Remitter Identification Number'
idCol2 = 'NR Transaction Details.Beneficiary Remitter Identification Number'
for i in ['Other Payments', 'Other Receipts']:
    for year in np.arange(2017, 2021):
        for sector in entitiesDict2:
            #match idCol1
            dfTemp = dfITIS2[(dfITIS2[transTypeCol] == i) &
                           (dfITIS2[yearCol] == year) &
                           (dfITIS2[idCol1].astype(str).isin(entitiesDict2[sector]))].copy()
            sumAmount = dfTemp[amountCol].sum()
            noTrans = dfTemp.shape[0]
            #store result: Sector, Receipts/Payments, Year, TotalAmount, NoTrans
            tempList = [sector, i, year, sumAmount, noTrans]
            resultList2.append(tempList)
            
            #match idCol2
            dfTemp = dfITIS2[(dfITIS2[transTypeCol] == i) &
                           (dfITIS2[yearCol] == year) &
                           (dfITIS2[idCol2].astype(str).isin(entitiesDict2[sector]))].copy()
            sumAmount = dfTemp[amountCol].sum()
            noTrans = dfTemp.shape[0]
            #store result: Sector, Receipts/Payments, Year, TotalAmount, NoTrans
            tempList = [sector, i, year, sumAmount, noTrans]
            resultList3.append(tempList)

In [74]:
dfResult2 = pd.DataFrame(resultList2)
dfResult2.columns = ["Sector", "TransactionType", "Year", "TotalAmount", "NoTrans"]
dfResult2['TransactionType'] = dfResult2['TransactionType'].map(lambda x: 'Receipts' if x == "Other Receipts"
                                                                           else "Payments")
dfResult2

,Sector,TransactionType,Year,TotalAmount,NoTrans
0,Banks,Payments,2017,3.114413e+09,98
1,Insurance,Payments,2017,1.592778e+09,4231
2,Leasing,Payments,2017,4.394661e+08,520
3,LITC,Payments,2017,2.336071e+09,325
4,Money Broker,Payments,2017,2.062955e+08,1810
...,...,...,...,...,...
83,Trust Company,Receipts,2020,1.735292e+08,3535
84,Foundations,Receipts,2020,1.007082e+07,28
85,Labuan Companies,Receipts,2020,2.881712e+10,34792
86,Partnerships,Receipts,2020,2.101274e+07,331


In [69]:
csvFile = "data/interim/2021-01-30_ItisEsTempTable2.csv"
dfResult2.to_csv(csvFile, index = False)

In [75]:
dfResult3 = pd.DataFrame(resultList3)
dfResult3.columns = ["Sector", "TransactionType", "Year", "TotalAmount", "NoTrans"]
dfResult3['TransactionType'] = dfResult3['TransactionType'].map(lambda x: 'Receipts' if x == "Other Receipts"
                                                                           else "Payments")
dfResult3

,Sector,TransactionType,Year,TotalAmount,NoTrans
0,Banks,Payments,2017,1.346384e+09,269
1,Insurance,Payments,2017,6.816710e+07,143
2,Leasing,Payments,2017,4.547483e+07,33
3,LITC,Payments,2017,6.626267e+07,15
4,Money Broker,Payments,2017,1.205530e+07,66
...,...,...,...,...,...
83,Trust Company,Receipts,2020,3.187438e+07,1275
84,Foundations,Receipts,2020,0.000000e+00,0
85,Labuan Companies,Receipts,2020,6.584516e+08,888
86,Partnerships,Receipts,2020,0.000000e+00,0


In [99]:
csvFile = "data/interim/2021-01-30_ItisEsTempTable3.csv"
dfResult3.to_csv(csvFile, index = False)

In [91]:
sumResult = []
#sum up result1 and result2
uniqueSet = dfResult2[["Sector", "TransactionType", "Year"]].values
for i in uniqueSet:
    sumAmount = dfResult.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['TotalAmount'].values[0]\
                + dfResult2.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['TotalAmount'].values[0]
    sumTrans = dfResult.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['NoTrans'].values[0]\
                + dfResult2.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['NoTrans'].values[0]
    tempList = [i[0], i[1], i[2], sumAmount, sumTrans]
    sumResult.append(tempList)

In [93]:
dfSumResult = pd.DataFrame(sumResult)
dfSumResult.columns = ["Sector", "TransactionType", "Year", "TotalAmount", "NoTrans"]
dfSumResult

,Sector,TransactionType,Year,TotalAmount,NoTrans
0,Banks,Payments,2017,3.114413e+09,98
1,Insurance,Payments,2017,1.595184e+09,4233
2,Leasing,Payments,2017,4.394661e+08,520
3,LITC,Payments,2017,2.336071e+09,325
4,Money Broker,Payments,2017,2.062955e+08,1810
...,...,...,...,...,...
83,Trust Company,Receipts,2020,1.735292e+08,3535
84,Foundations,Receipts,2020,1.007082e+07,28
85,Labuan Companies,Receipts,2020,2.881712e+10,34792
86,Partnerships,Receipts,2020,2.101274e+07,331


In [98]:
#concat 2016 from dfResult and then dfSumResult (from 2017 onwards)
dfResult2016 = dfResult.query("Year == 2016")
dfCombined = pd.concat([dfResult2016, dfSumResult]).copy()

In [100]:
csvFile = "data/interim/2021-01-30_ItisEsTempTableCombined.csv"
dfCombined.to_csv(csvFile, index = False)

In [103]:
idList = [j for i in entitiesDict2.values() for j in i]

In [111]:
idCol1 = 'NR Customer Details.Customer Beneficiary Remitter Identification Number'
idCol2 = 'NR Transaction Details.Beneficiary Remitter Identification Number'

indexList1 = dfITIS2[dfITIS2[idCol1].astype(str).isin(entityIdList)].index
indexList2 = dfITIS2[dfITIS2[idCol2].astype(str).isin(entityIdList)].index

In [109]:
dfTemp = dfITIS2[(dfITIS2[idCol1].astype(str).isin(entityIdList)) &
                (dfITIS2[idCol2].astype(str).isin(entityIdList))].copy()

In [110]:
dfTemp.shape

(20683, 16)

In [113]:
dfTemp.head()

,NR - MYR Amount,Country.Country Description,NR Customer Details.Customer Beneficiary Remitter Account Number,NR Customer Details.Customer Beneficiary Remitter Identification Number,NR Customer Details.Customer Beneficiary Remitter Name,NR Transaction Details.Beneficiary Remitter Identification Number,NR Transaction Details.Name of Beneficiary Remitter,NR Transaction Details.Transaction Date,NR Transaction Details.Transaction Status,NR Transaction Details.Transaction Type,Reporting Entity.RE Name,Reporting Entity.RE Name 2,Reporting Period.Reporting Quarter,Reporting Period.Reporting Year,Purpose Code.Purpose Code Description,Reporting Period.Reporting Month
13711,806802.00,LABUAN OFFSHORE,NaN,LL10729,APDIB HOLDING LIMITED,LL09362,INTERACT PERFORMANCE LTD,2017-08-10 00:00:00,COMPLETE,Other Payments,BANK OF CHINA (MALAYSIA) BERHAD,BANK OF CHINA (MALAYSIA) BERHAD,Q3,2017,Unknown,NaN
13767,435.24,Netherlands,NaN,LL11399,SPLENDID INVESTMENTS LIMITED,LL11399,SPLENDID INVESTMENT LIMITED,2017-08-15 00:00:00,COMPLETE,Other Payments,BNP PARIBAS MALAYSIA BERHAD,BNP PARIBAS MALAYSIA BERHAD,Q3,2017,Unknown,NaN
13903,17203246.00,United States,NaN,LL07814,MFI LABUAN LTD,LL07814,MFI LABUAN LTD,2017-07-13 00:00:00,COMPLETE,Other Payments,UNITED OVERSEAS BANK (MALAYSIA) BHD.,UNITED OVERSEAS BANK (MALAYSIA) BHD.,Q3,2017,Unknown,NaN
13904,128565.00,United States,NaN,LL08001,ANGLO-EASTERN (LABUAN) LIMITED,LL08001,ANGLO-EASTERN (LABUAN) LIMITED,2017-07-19 00:00:00,COMPLETE,Other Receipts,BNP PARIBAS MALAYSIA BERHAD,BNP PARIBAS MALAYSIA BERHAD,Q3,2017,Unknown,NaN
13922,47000.44,Australia,NaN,LL12954,FASA CENTRA SOLUTIONS LTD,LL12954,INTERNATIONAL CAPITAL MARKETS PTY LTD,2017-07-25 00:00:00,COMPLETE,Other Payments,RHB BANK BERHAD,RHB BANK BERHAD,Q3,2017,Unknown,NaN


In [116]:
[i for i,j in entitiesDict.items() if 'LL10729' in j]

['Labuan Companies']

In [117]:
[i for i,j in entitiesDict.items() if 'LL09362' in j]

['Labuan Companies']

In [118]:
sumResult2 = []
#sum up result1, result2 and result3
uniqueSet = dfResult2[["Sector", "TransactionType", "Year"]].values
for i in uniqueSet:
    sumAmount = dfResult.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['TotalAmount'].values[0]\
                + dfResult2.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['TotalAmount'].values[0]\
             + dfResult3.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['TotalAmount'].values[0]
    sumTrans = dfResult.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['NoTrans'].values[0]\
                + dfResult2.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['NoTrans'].values[0]\
                + dfResult3.query("Sector == '%s' & TransactionType == '%s' & Year == %s"%(i[0], i[1], i[2]))['NoTrans'].values[0]
    tempList = [i[0], i[1], i[2], sumAmount, sumTrans]
    sumResult2.append(tempList)

In [119]:
dfSumResult2 = pd.DataFrame(sumResult2)
dfSumResult2.columns = ["Sector", "TransactionType", "Year", "TotalAmount", "NoTrans"]
dfSumResult2

,Sector,TransactionType,Year,TotalAmount,NoTrans
0,Banks,Payments,2017,4.460797e+09,367
1,Insurance,Payments,2017,1.663351e+09,4376
2,Leasing,Payments,2017,4.849410e+08,553
3,LITC,Payments,2017,2.402333e+09,340
4,Money Broker,Payments,2017,2.183508e+08,1876
...,...,...,...,...,...
83,Trust Company,Receipts,2020,2.054035e+08,4810
84,Foundations,Receipts,2020,1.007082e+07,28
85,Labuan Companies,Receipts,2020,2.947557e+10,35680
86,Partnerships,Receipts,2020,2.101274e+07,331


In [120]:
#concat 2016 from dfResult and then dfSumResult (from 2017 onwards)
dfResult2016 = dfResult.query("Year == 2016")
dfCombined2 = pd.concat([dfResult2016, dfSumResult2]).copy()

In [121]:
csvFile = "data/interim/2021-02-02_ItisEsTempTableCombined2.csv"
dfCombined2.to_csv(csvFile, index = False)

In [122]:
dfCombined2

,Sector,TransactionType,Year,TotalAmount,NoTrans
0,Banks,Receipts,2016,0.000000e+00,0
1,Insurance,Receipts,2016,0.000000e+00,0
2,Leasing,Receipts,2016,0.000000e+00,0
3,LITC,Receipts,2016,2.220020e+08,22
4,Money Broker,Receipts,2016,0.000000e+00,0
...,...,...,...,...,...
83,Trust Company,Receipts,2020,2.054035e+08,4810
84,Foundations,Receipts,2020,1.007082e+07,28
85,Labuan Companies,Receipts,2020,2.947557e+10,35680
86,Partnerships,Receipts,2020,2.101274e+07,331


### Write Result

In [21]:
excelFile = "data/interim/2021-01-29_ItisSummaryLabuanFSA.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfFinal1.to_excel(writer, sheet_name = 'NoTrans')
    dfFinal2.to_excel(writer, sheet_name = 'TotalAmount')